<a href="https://colab.research.google.com/github/RochaErik/AlgorithmComparison/blob/main/AlgorithmComparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [214]:
!pip install catboost
!pip install lightgbm
!pip install xgboost

In [215]:
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

# **(Unbalanced) Wine Dataset**

In [216]:
wine_df = pd.read_csv('/content/drive/MyDrive/DatasetSeminario/Wine/wine.data', header=None)

In [217]:
wine_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [218]:
wine_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       178 non-null    int64  
 1   1       178 non-null    float64
 2   2       178 non-null    float64
 3   3       178 non-null    float64
 4   4       178 non-null    float64
 5   5       178 non-null    int64  
 6   6       178 non-null    float64
 7   7       178 non-null    float64
 8   8       178 non-null    float64
 9   9       178 non-null    float64
 10  10      178 non-null    float64
 11  11      178 non-null    float64
 12  12      178 non-null    float64
 13  13      178 non-null    int64  
dtypes: float64(11), int64(3)
memory usage: 19.6 KB


In [219]:
wine_df.nunique()

0       3
1     126
2     133
3      79
4      63
5      53
6      97
7     132
8      39
9     101
10    132
11     78
12    122
13    121
dtype: int64

In [220]:
# Unbalanced dataset

wine_df[0].value_counts()

2    71
1    59
3    48
Name: 0, dtype: int64

In [221]:
wine_df.isna().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
dtype: int64

In [222]:
X = wine_df.iloc[:, 1:]
y = wine_df.iloc[:, 0]

In [223]:
X.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [224]:
X.shape

(178, 13)

In [225]:
y.head()

0    1
1    1
2    1
3    1
4    1
Name: 0, dtype: int64

In [226]:
y.shape

(178,)

In [227]:
# Since version 1.3.2 XGBoost needs target columns to start with 0 value

le = LabelEncoder()
y = le.fit_transform(y)

In [228]:
names = [
          'AdaBoost',
          'CatBoost',
          'LightGBM',
          'XGBoost'
        ]

classifiers = [
                AdaBoostClassifier(),
                CatBoostClassifier(silent=True),
                LGBMClassifier(verbosity=-1),
                XGBClassifier()
              ]

In [229]:
rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=42)

In [230]:
# wine_scores_mean = []
# wine_scores_std = []

# for name, clf in zip(names, classifiers):
#   results = cross_val_score(clf, X, y, cv=rkf)
#   wine_scores_mean.append(results.mean()*100)
#   wine_scores_std.append(results.std()*100)
#   print(f'--------- {name} on Wine Dataset ---------')
#   print(results)
#   print('Accuracy: %.2f%% (%.2f%%)' % (results.mean()*100, results.std()*100))
#   print('------------------------------')

In [231]:
# wine_scores_mean

In [232]:
# wine_scores_std

In [233]:
# Algo_results = pd.DataFrame()
# Algo_results['Name'] = names
# Algo_results['Wine_Results'] = wine_scores_mean

In [234]:
# Algo_results

# **(Unbalanced) Breast Cancer Dataset**

In [235]:
breast_cancer_df = pd.read_csv('/content/drive/MyDrive/DatasetSeminario/BreastCancer/breastCancer.data', header=None, na_values='?')

In [236]:
breast_cancer_df.head()

,0,1,2,3,4,5,6,7,8,9
0,no-recurrence-events,30-39,premeno,30-34,0-2,no,3,left,left_low,no
1,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,right,right_up,no
2,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,left,left_low,no
3,no-recurrence-events,60-69,ge40,15-19,0-2,no,2,right,left_up,no
4,no-recurrence-events,40-49,premeno,0-4,0-2,no,2,right,right_low,no


In [237]:
print(breast_cancer_df.shape)

(286, 10)


In [238]:
column_names = ['Class', 'age', 'menopause', 'tumor_size', 'inv_nodes', 'node_caps', 'deg_malig', 'breast', 'breast_quad', 'irradiat']
breast_cancer_df.columns = column_names

In [239]:
breast_cancer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286 entries, 0 to 285
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Class        286 non-null    object
 1   age          286 non-null    object
 2   menopause    286 non-null    object
 3   tumor_size   286 non-null    object
 4   inv_nodes    286 non-null    object
 5   node_caps    278 non-null    object
 6   deg_malig    286 non-null    int64 
 7   breast       286 non-null    object
 8   breast_quad  285 non-null    object
 9   irradiat     286 non-null    object
dtypes: int64(1), object(9)
memory usage: 22.5+ KB


In [240]:
breast_cancer_df.nunique()

Class           2
age             6
menopause       3
tumor_size     11
inv_nodes       7
node_caps       2
deg_malig       3
breast          2
breast_quad     5
irradiat        2
dtype: int64

In [241]:
# Unbalanced dataset

breast_cancer_df['Class'].value_counts()

no-recurrence-events    201
recurrence-events        85
Name: Class, dtype: int64

In [242]:
breast_cancer_df.isna().sum()

Class          0
age            0
menopause      0
tumor_size     0
inv_nodes      0
node_caps      8
deg_malig      0
breast         0
breast_quad    1
irradiat       0
dtype: int64

In [243]:
# Checking the proportion of NaN values

missing_count = breast_cancer_df['node_caps'].isna().sum()
total_count = len(breast_cancer_df['node_caps'])
missing_ratio = missing_count / total_count
missing_ratio

0.027972027972027972

In [244]:
# The proportion of NaN values was low so we opt to remove the rows

breast_cancer_df.dropna(inplace=True)

In [245]:
breast_cancer_df.isna().sum()

Class          0
age            0
menopause      0
tumor_size     0
inv_nodes      0
node_caps      0
deg_malig      0
breast         0
breast_quad    0
irradiat       0
dtype: int64

In [246]:
age_dummies = {
                '10-19': 0,
                '20-29': 1,
                '30-39': 2,
                '40-49': 3,
                '50-59': 4,
                '60-69': 5,
                '70-79': 6,
                '80-89': 7,
                '90-99': 8
              }
breast_cancer_df['age'].replace(age_dummies, inplace=True)

In [247]:
breast_cancer_df['age']

0      2
1      3
2      3
3      5
4      3
      ..
281    2
282    2
283    5
284    3
285    4
Name: age, Length: 277, dtype: int64

In [248]:
size_dummies = {
                '0-4': 0,
                '5-9': 1,
                '10-14': 2,
                '15-19': 3,
                '20-24': 4,
                '25-29': 5,
                '30-34': 6,
                '35-39': 7,
                '40-44': 8,
                '45-49': 9,
                '50-54': 10,
                '55-59': 11
                }
breast_cancer_df['tumor_size'].replace(size_dummies, inplace=True)

In [249]:
breast_cancer_df['tumor_size']

0      6
1      4
2      4
3      3
4      0
      ..
281    6
282    4
283    4
284    6
285    6
Name: tumor_size, Length: 277, dtype: int64

In [250]:
invnodes_dummies = {
                '0-2': 0,
                '3-5': 1,
                '6-8': 2,
                '9-11': 3,
                '12-14': 4,
                '15-17': 5,
                '18-20': 6,
                '21-23': 7,
                '24-26': 8,
                '27-29': 9,
                '30-32': 10,
                '33-35': 11,
                '36-39': 12,
              }
breast_cancer_df['inv_nodes'].replace(invnodes_dummies, inplace=True)

In [251]:
breast_cancer_df['inv_nodes']

0      0
1      0
2      0
3      0
4      0
      ..
281    0
282    0
283    0
284    1
285    1
Name: inv_nodes, Length: 277, dtype: int64

In [252]:
# Important: This isnt recommended for CatBoost. One-hot encoding during preprocessing affects both the training speed and
# the resulting quality.

dummies = pd.get_dummies(breast_cancer_df[['menopause', 'node_caps', 'breast', 'breast_quad', 'irradiat']], drop_first=True)
breast_cancer_df = pd.concat([breast_cancer_df, dummies], axis=1)
breast_cancer_df.head(5)

,Class,age,menopause,tumor_size,inv_nodes,node_caps,deg_malig,breast,breast_quad,irradiat,menopause_lt40,menopause_premeno,node_caps_yes,breast_right,breast_quad_left_low,breast_quad_left_up,breast_quad_right_low,breast_quad_right_up,irradiat_yes
0,no-recurrence-events,2,premeno,6,0,no,3,left,left_low,no,0,1,0,0,1,0,0,0,0
1,no-recurrence-events,3,premeno,4,0,no,2,right,right_up,no,0,1,0,1,0,0,0,1,0
2,no-recurrence-events,3,premeno,4,0,no,2,left,left_low,no,0,1,0,0,1,0,0,0,0
3,no-recurrence-events,5,ge40,3,0,no,2,right,left_up,no,0,0,0,1,0,1,0,0,0
4,no-recurrence-events,3,premeno,0,0,no,2,right,right_low,no,0,1,0,1,0,0,1,0,0


In [253]:
breast_cancer_df.drop(['menopause', 'node_caps', 'breast', 'breast_quad', 'irradiat'], axis=1, inplace=True)

In [254]:
breast_cancer_df.head(5)

,Class,age,tumor_size,inv_nodes,deg_malig,menopause_lt40,menopause_premeno,node_caps_yes,breast_right,breast_quad_left_low,breast_quad_left_up,breast_quad_right_low,breast_quad_right_up,irradiat_yes
0,no-recurrence-events,2,6,0,3,0,1,0,0,1,0,0,0,0
1,no-recurrence-events,3,4,0,2,0,1,0,1,0,0,0,1,0
2,no-recurrence-events,3,4,0,2,0,1,0,0,1,0,0,0,0
3,no-recurrence-events,5,3,0,2,0,0,0,1,0,1,0,0,0
4,no-recurrence-events,3,0,0,2,0,1,0,1,0,0,1,0,0


In [255]:
X = breast_cancer_df.iloc[:, 1:]
y = breast_cancer_df.iloc[:, 0]

In [256]:
X.head()

,age,tumor_size,inv_nodes,deg_malig,menopause_lt40,menopause_premeno,node_caps_yes,breast_right,breast_quad_left_low,breast_quad_left_up,breast_quad_right_low,breast_quad_right_up,irradiat_yes
0,2,6,0,3,0,1,0,0,1,0,0,0,0
1,3,4,0,2,0,1,0,1,0,0,0,1,0
2,3,4,0,2,0,1,0,0,1,0,0,0,0
3,5,3,0,2,0,0,0,1,0,1,0,0,0
4,3,0,0,2,0,1,0,1,0,0,1,0,0


In [257]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 277 entries, 0 to 285
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   age                    277 non-null    int64
 1   tumor_size             277 non-null    int64
 2   inv_nodes              277 non-null    int64
 3   deg_malig              277 non-null    int64
 4   menopause_lt40         277 non-null    uint8
 5   menopause_premeno      277 non-null    uint8
 6   node_caps_yes          277 non-null    uint8
 7   breast_right           277 non-null    uint8
 8   breast_quad_left_low   277 non-null    uint8
 9   breast_quad_left_up    277 non-null    uint8
 10  breast_quad_right_low  277 non-null    uint8
 11  breast_quad_right_up   277 non-null    uint8
 12  irradiat_yes           277 non-null    uint8
dtypes: int64(4), uint8(9)
memory usage: 13.3 KB


In [258]:
y.head()

0    no-recurrence-events
1    no-recurrence-events
2    no-recurrence-events
3    no-recurrence-events
4    no-recurrence-events
Name: Class, dtype: object

In [259]:
# Since version 1.3.2 XGBoost needs target columns to start with 0 value

le = LabelEncoder()
y = le.fit_transform(y)

In [260]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [261]:
# breast_cancer_mean = []
# breast_cancer_std = []

# for name, clf in zip(names, classifiers):
#   results = cross_val_score(clf, X, y, cv=rkf)
#   breast_cancer_mean.append(results.mean()*100)
#   breast_cancer_std.append(results.std()*100)
#   print(f'--------- {name} on Breast Cancer Dataset ---------')
#   print(results)
#   print('Accuracy: %.2f%% (%.2f%%)' % (results.mean()*100, results.std()*100))
#   print('------------------------------')

In [262]:
breast_cancer_mean

[71.52116402116403, 73.64285714285715, 71.44841269841271, 69.94708994708995]

In [263]:
breast_cancer_std

[8.876449866016364, 8.066671160861826, 7.933561512153653, 8.513464273378986]

In [265]:
# Algo_results['Breast_Cancer_Results'] = breast_cancer_mean

# **Sonar Dataset** #

In [266]:
sonar_df = pd.read_csv('/content/drive/MyDrive/DatasetSeminario/Sonar/Sonar.csv', header=None)

In [267]:
sonar_df.shape

(208, 61)

In [268]:
sonar_df.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


In [269]:
sonar_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208 entries, 0 to 207
Data columns (total 61 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       208 non-null    float64
 1   1       208 non-null    float64
 2   2       208 non-null    float64
 3   3       208 non-null    float64
 4   4       208 non-null    float64
 5   5       208 non-null    float64
 6   6       208 non-null    float64
 7   7       208 non-null    float64
 8   8       208 non-null    float64
 9   9       208 non-null    float64
 10  10      208 non-null    float64
 11  11      208 non-null    float64
 12  12      208 non-null    float64
 13  13      208 non-null    float64
 14  14      208 non-null    float64
 15  15      208 non-null    float64
 16  16      208 non-null    float64
 17  17      208 non-null    float64
 18  18      208 non-null    float64
 19  19      208 non-null    float64
 20  20      208 non-null    float64
 21  21      208 non-null    float64
 22  22

In [270]:
# A little unbalanced dataset

sonar_df[60].value_counts()

M    111
R     97
Name: 60, dtype: int64

In [271]:
X = sonar_df.iloc[:, :-1]
y = sonar_df.iloc[:, -1]

In [272]:
X.head()

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0232,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0125,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0033,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0241,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0156,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094


In [273]:
y.head()

0    R
1    R
2    R
3    R
4    R
Name: 60, dtype: object

In [274]:
# Since version 1.3.2 XGBoost needs target columns to start with 0 value

le = LabelEncoder()
y = le.fit_transform(y)

In [275]:
sonar_mean = []
sonar_std = []

for name, clf in zip(names, classifiers):
  results = cross_val_score(clf, X, y, cv=rkf)
  sonar_mean.append(results.mean()*100)
  sonar_std.append(results.std()*100)
  print(f'--------- {name} on Sonar Dataset ---------')
  print(results)
  print('Accuracy: %.2f%% (%.2f%%)' % (results.mean()*100, results.std()*100))
  print('------------------------------')

--------- AdaBoost on Sonar Dataset ---------
[0.9047619  0.76190476 0.76190476 0.76190476 0.85714286 0.80952381
 0.9047619  0.85714286 0.9        0.8        0.80952381 0.85714286
 0.57142857 0.76190476 0.80952381 0.85714286 0.85714286 0.80952381
 0.75       0.85       0.9047619  0.85714286 0.9047619  0.76190476
 0.85714286 0.85714286 0.85714286 0.80952381 0.8        0.9
 0.76190476 0.85714286 0.80952381 0.9047619  0.85714286 0.80952381
 0.76190476 0.71428571 0.75       0.95       0.76190476 0.80952381
 0.76190476 0.95238095 0.9047619  0.80952381 0.95238095 0.85714286
 0.8        0.8        0.9047619  0.9047619  0.9047619  0.66666667
 0.76190476 0.80952381 0.9047619  0.85714286 0.9        0.95
 0.9047619  0.85714286 0.85714286 0.9047619  0.9047619  0.80952381
 0.85714286 0.76190476 0.85       0.7        0.66666667 0.95238095
 0.80952381 0.66666667 0.76190476 0.9047619  0.71428571 0.9047619
 1.         0.95       0.61904762 0.9047619  0.85714286 0.9047619
 0.80952381 0.85714286 0.714285

In [276]:
sonar_mean

[83.18809523809522, 87.15238095238097, 88.12380952380954, 85.00714285714285]

In [277]:
sonar_std

[7.845438251655812, 8.016770969663867, 7.246883762213835, 8.24734238785956]